In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
#from vsopy import util, data
import astropy.units as u


In [ ]:
import vsopy.util
import vsopy.data
from astropy.table import QTable

OBJ_NAME='SZ Lyn'
SESSION_TAG='20250122'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vsopy.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vsopy.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vsopy.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir / 'V'
sd = vsopy.data.StarData(layout.charts_dir)

movie_dir = session_layout.root_dir / 'movie'
if not movie_dir.exists():
    movie_dir.mkdir(parents=True)

chart = sd.get_chart(OBJ_NAME, 60*u.arcmin)
stars = QTable(dict(auid = [chart.meta['auid']], sky_centroid= [chart.meta['radec2000']]))
stars

In [ ]:
%%capture
import ccdproc as ccdp
import vsopy.reduce
from astropy.visualization import AsinhStretch, ImageNormalize, MinMaxInterval
import matplotlib.pyplot as plt
from astropy.nddata import Cutout2D
import plot as vp
from astropy.coordinates import SkyCoord
from astropy.time import Time
import vsopy.phot

ifc = ccdp.ImageFileCollection(IMAGE_DIR)
alpha = 0.01

interval = MinMaxInterval()

for f in ifc.summary['file']:
    image = vsopy.reduce.load_and_solve(ifc.location / f, session_layout.solved_dir)
    fig = plt.figure(figsize=(10.24, 10.24))

    center = vsopy.phot.measure_photometry(image, stars, 5*u.arcsec, (10*u.arcsec, 15*u.arcsec))['sky_centroid'][0]
#    center = SkyCoord(ra=image.wcs.wcs.crval[0]*u.deg, dec=image.wcs.wcs.crval[1]*u.deg)
    cutout =  Cutout2D(image.data,
                       center,
                       (800, 1280),
                       wcs=image.wcs)
    #vmin, vmax = interval.get_limits(cutout.data)
    #print(vmin, vmax)
    ax = plt.subplot(projection=cutout.wcs)
    vmin, vmax = 1000, 65535
    norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=AsinhStretch(alpha))
    ax.imshow(cutout.data, origin='lower', norm=norm, cmap='gray')
    d = Time(image.header['DATE-OBS'])
    ax.text(20, 750, f'JD2000: {d.to_value('jd'):.3f}', color='white')

    ax.set_axis_off()
    ax.set_position([0, 0, 1, 1])
    #plt.show()
    plt.savefig(movie_dir / (f + '.jpg'), bbox_inches='tight', facecolor='black')


In [ ]:

image.header